In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from sqlalchemy import create_engine
import pymysql
import dotenv
dotenv.load_dotenv()
MYSQL_USER=os.getenv('MYSQL_USER')
MYSQL_PASSWORD=os.getenv('MYSQL_PASSWORD')

In [2]:
# Load csv into mysql database, in order to use sql query to analyze data
conn = create_engine("mysql+pymysql://" + MYSQL_USER + ":" + MYSQL_PASSWORD + "@localhost:3306/airflow_project")
df = pd.read_csv('AB_NYC_2019.csv', delimiter=',')
df.to_sql(name='nyc_abb', con=conn, schema='airflow_project', if_exists='replace')

In [3]:
engine = create_engine("mysql+pymysql://root:" + os.environ.get("MYSQL_PASSWORD") + '@localhost:3306/airflow_project')

In [4]:
abb_mysql_df = pd.read_sql('select * from airflow_project.nyc_abb', con=engine)
abb_mysql_df.head()

,index,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,None,NaN,1,365
3,3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


#### more detail data cleaning

In [5]:
abb_mysql_df.loc[abb_mysql_df['reviews_per_month'].isnull(), 'reviews_per_month'] = 0

In [6]:
abb_mysql_df = abb_mysql_df.loc[:, ~ abb_mysql_df.columns.str.contains('^Unnamed')]

In [7]:
abb_mysql_df = abb_mysql_df[abb_mysql_df['price']>0]

In [8]:
abb_mysql_df = abb_mysql_df[abb_mysql_df['minimum_nights']<=365]

In [9]:
abb_mysql_df = abb_mysql_df.set_index('host_id')

In [10]:
abb_mysql_df.drop(columns=['index']).head()

,id,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
host_id,,,,,,,,,,,,,,,
2787,2539,Clean & quiet apt home by the park,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
2845,2595,Skylit Midtown Castle,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
4632,3647,THE VILLAGE OF HARLEM....NEW YORK !,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,None,0.00,1,365
4869,3831,Cozy Entire Floor of Brownstone,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
7192,5022,Entire Apt: Spacious Studio/Loft by central park,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [11]:
abb_mysql_df.shape

(48870, 16)

In [12]:
abb_mysql_df.dtypes

index                               int64
id                                  int64
name                               object
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [13]:
abb_mysql_df.isnull().sum()

index                                 0
id                                    0
name                                 15
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10043
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
dtype: int64

### Example using SQL query to pull out data

In [14]:
# More Visualization from SQL query
connection = engine.connect()

In [15]:
# Average Price of Airbnb in NYC /night
avg_abb_price = connection.execute('SELECT AVG(price) FROM airflow_project.nyc_abb')

In [16]:
data_avg_abb_price = [row for row in avg_abb_price]
df_avg_abb_price = pd.DataFrame.from_records(data_avg_abb_price)
df_avg_abb_price.columns = ['Average price/night of Airbnb in NYC 2019']
df_avg_abb_price

,Average price/night of Airbnb in NYC 2019
0,152.7207


In [17]:
# Average Price of Airbnb in NYC /night (Manhattan) 
avg_abb_price_M = connection.execute('SELECT AVG(price) \
                                     FROM airflow_project.nyc_abb \
                                      WHERE neighbourhood_group = "Manhattan"')

In [18]:
data_avg_abb_price_M = [row for row in avg_abb_price_M]
df_avg_abb_price_M = pd.DataFrame.from_records(data_avg_abb_price_M)
df_avg_abb_price_M.columns = ['Average price/night of Airbnb in NYC 2019 (Manhattan)']
df_avg_abb_price_M

,Average price/night of Airbnb in NYC 2019 (Manhattan)
0,196.8758


In [19]:
# Average Price of Airbnb in NYC /night (Queens) 
avg_abb_price_Q = connection.execute('SELECT AVG(price) \
                                     FROM airflow_project.nyc_abb\
                                     WHERE neighbourhood_group = "Queens"')

In [20]:
data_avg_abb_price_Q = [row for row in avg_abb_price_Q]
df_avg_abb_price_Q = pd.DataFrame.from_records(data_avg_abb_price_Q)
df_avg_abb_price_Q.columns = ['Average price/night of Airbnb in NYC 2019 (Queens)']
df_avg_abb_price_Q

,Average price/night of Airbnb in NYC 2019 (Queens)
0,99.5176


In [21]:
#Total Number of review
sum_abb_reviews = connection.execute('SELECT SUM(number_of_reviews)\
                                        FROM airflow_project.nyc_abb')

In [22]:
data_sum_abb_reviews = [row for row in sum_abb_reviews]
df_sum_abb_reviews = pd.DataFrame.from_records(data_sum_abb_reviews)
df_sum_abb_reviews.columns = ['Total nunber of reviews for Airbnb housing in NYC 2019']
df_sum_abb_reviews

,Total nunber of reviews for Airbnb housing in NYC 2019
0,1138005


### Next, we will look at the price distribution by the different boroughs

In [23]:
abb_mysql_df.neighbourhood_group.unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [24]:
set(abb_mysql_df['neighbourhood_group'])

{'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'}

In [25]:
abb_mysql_df.groupby('neighbourhood_group')['price'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
neighbourhood_group,,,,,,,,
Bronx,1090.0,87.58,106.73,10.0,45.0,65.0,99.0,2500.0
Brooklyn,20089.0,124.45,186.92,10.0,60.0,90.0,150.0,10000.0
Manhattan,21654.0,196.89,291.42,10.0,95.0,150.0,220.0,10000.0
Queens,5664.0,99.49,167.13,10.0,50.0,75.0,110.0,10000.0
Staten Island,373.0,114.81,277.62,13.0,50.0,75.0,110.0,5000.0


In [26]:
#The minimum nights for each borough
abb_mysql_df.groupby('neighbourhood_group')['minimum_nights'].describe()

,count,mean,std,min,25%,50%,75%,max
neighbourhood_group,,,,,,,,
Bronx,1090.0,4.563303,15.638775,1.0,1.0,2.0,3.0,365.0
Brooklyn,20089.0,5.894569,14.533385,1.0,2.0,3.0,5.0,365.0
Manhattan,21654.0,8.345617,18.824170,1.0,1.0,3.0,6.0,365.0
Queens,5664.0,5.010240,11.952636,1.0,1.0,2.0,3.0,365.0
Staten Island,373.0,4.831099,19.727605,1.0,1.0,2.0,3.0,365.0


In [27]:
#Pivot table, 5 boroughs
abb_mysql_df_pt = abb_mysql_df.pivot_table(index='neighbourhood_group', columns='room_type', values='price', aggfunc='mean')
abb_mysql_df_pt.round(2)

room_type,Entire home/apt,Private room,Shared room
neighbourhood_group,,,
Bronx,127.51,66.89,59.80
Brooklyn,178.36,76.55,50.77
Manhattan,249.28,116.78,88.93
Queens,147.03,71.76,69.02
Staten Island,173.85,62.29,57.44


### Now, let's do simle combing dataset set by couting data from 5 boroughs

#### NYC Airbnb Housing Data Count Summary

In [28]:
abb_borough_count = abb_mysql_df.groupby('neighbourhood_group').neighbourhood_group.count()
abb_borough_count

neighbourhood_group
Bronx             1090
Brooklyn         20089
Manhattan        21654
Queens            5664
Staten Island      373
Name: neighbourhood_group, dtype: int64

In [29]:
# SELECT TOP 35 neighbourhood, count(*) FROM nyc_abb
# GROUP BY neighbourhood_group
# ORDER BY COUNT(*) DESC
abb_mysql_df_top35 = connection.execute('SELECT neighbourhood_group, COUNT(*) \
                                     FROM airflow_project.nyc_abb \
                                       GROUP BY neighbourhood_group \
                                     ORDER BY COUNT(*) DESC')

In [30]:
abb_mysql_df_top35 = [row for row in abb_mysql_df_top35]
abb_mysql_df_top35 = pd.DataFrame.from_records(abb_mysql_df_top35)
abb_mysql_df_top35.columns = [['Borough', 'Count']]
abb_mysql_df_top35

,Borough,Count
0,Manhattan,21661
1,Brooklyn,20104
2,Queens,5666
3,Bronx,1091
4,Staten Island,373


In [31]:
# Wrap a function to implement the sql

abb_mysql_df_top35 = connection.execute('SELECT neighbourhood_group, COUNT(*) \
                                    FROM airflow_project.nyc_abb \
                                    GROUP BY neighbourhood_group \
                                    ORDER BY COUNT(*) DESC')

In [32]:
def sql_query_to_pddf(sql_query, *argv):
    sql_query = [row for row in sql_query]
    sql_query = pd.DataFrame.from_records(sql_query)
    sql_query.columns = [[*argv]]
    return sql_query

In [33]:
sql_query_to_pddf(abb_mysql_df_top35, 'Borough', 'Count')

,Borough,Count
0,Manhattan,21661
1,Brooklyn,20104
2,Queens,5666
3,Bronx,1091
4,Staten Island,373
